<a href="https://colab.research.google.com/github/Scjatto/PyCollab/blob/main/Sonifiication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Basic Libraries**

In [1]:
import os
import pandas as pd

# **Getting Data from personal Drive Folder**




*   DataSet path taken from personal Drive
*   Drive mounted to this notebook to access drive folders
*   Source of data will be different in different systems





In [4]:
current_working_dir = os.getcwd()
# $(PWD)/drive/MyDrive/DataSets/Sonification
data_set_folder_name = os.path.join("drive", "MyDrive", "DataSets", "Sonification")
data_set_file_name = "lunarCraterAges.csv"
data_set_path = os.path.join(current_working_dir, data_set_folder_name, data_set_file_name) # Path of csv data file

In [5]:
df = pd.read_csv(data_set_path)  #load data as a pandas dataframe
df.head(5) #take a look at first 5 rows

,names,longitude,latitude,diameter,age
0,Copernicus,339.92969,9.63280,96.7,797
1,Tycho,348.71469,-43.25890,85.7,85
2,King,120.49220,4.93750,76.2,992
3,Jackson,196.68950,22.04000,71.4,147
4,Ohm,246.27397,18.28491,64.3,291
